Solution 1: building model from scratch wothout transfer learning - Val_accuracy: 67.1% - test accuracy: 

parameters:
7 conv layaers 
5 Dense layers - relu activation  
1 softmax 

optimizer: ADAM with lr = 0.0001
dropout: 40% 
Augmentation: yes
class weights added to compensate for data imbalance

note: i have a problem with result reproducibility, I have fixed a seed for tensorflow, keras and used keras intitilaiztion in the dense layers. but the results differ a little bit for run to another. I think this randomness comes from cloud GPU. 




Impoting data from googe drive and importing the useful libraries

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
base_dir = "/content/drive/My Drive/classification/classification/Classification_Dataset/training"

In [3]:
#try:
  # Use the %tensorflow_version magic if in colab.
  #%tensorflow_version 2.x
#except Exception:
  #pass

import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [4]:
import os
import cv2
import glob
import shutil
import numpy as np
import keras
from keras.applications.inception_v3 import InceptionV3
from keras import backend as K
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.models import Model
from keras.layers import Dense,GlobalAveragePooling2D,Dropout,SeparableConv2D, BatchNormalization, Activation,  Conv2D, Flatten, MaxPooling2D
from keras.applications.mobilenet import MobileNet
from keras.optimizers import Adam, SGD
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from sklearn.datasets import load_files  # to load the images from the subfolders    
from keras.utils import np_utils # to hot encode the label of the image
import matplotlib.pyplot as plt # to draw barplot and etc...
from tqdm import tqdm 
from keras.preprocessing import image    
from keras.callbacks import ModelCheckpoint # to save the best weights for the model while training 
import random as rn
from sklearn.utils import class_weight
from datetime import datetime
import cv2
from PIL import Image


Using TensorFlow backend.


Fixing a random seed to make the results reproducible

In [0]:
rand_seed = 19
os.environ['PYTHONHASHSEED'] = '0'

#random seed for NP genreator of ranodm numbers
np.random.seed(rand_seed)

#random seed generator for Python
rn.seed(rand_seed)

#random seed for tensorflow
tf.set_random_seed(rand_seed)

session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

defining the classes in the correct order {owl: 0, galaxy: 2, ... 

In [0]:
class_list = [
              'owl',
              'galaxy',
              'lightning',
              'wine-bottle',
              't-shirt',
              'waterfall',
              'sword',
              'school-bus',
              'calculator',
              'sheet-music',
              'airplanes',
              'lightbulb',
              'skyscraper',
              'mountain-bike',
              'fireworks',
              'computer-monitor',
              'bear',
              'grand-piano',
              'kangaroo',
              'laptop'
              ]

 since the data is not splitted into training and vailation sets. i will do so in the next code .The code below creates a train and a val folder each containing 20 folders (one for each type of class). It then moves the images from the original folders to these new folders such that 80% of the images go to the training set and 20% of the images go into the validation set. In the end our directory will have the 

Since I did nott delete the original folders, they will still be in the directory which is ok 

In [7]:
train_list = []
val_list = []
train_dict = {}
val_dict = {}

for cl in class_list:
  img_path = os.path.join(base_dir, cl)
  #print(img_path)
  images = glob.glob(img_path + '/*')
  #print("{}: {} Images".format(cl, len(images)))
  num_train = int(round(len(images)*0.8))
  train, val = images[:num_train], images[num_train:]

  for t in train:
    if not os.path.exists(os.path.join(base_dir, 'train', cl)):
      os.makedirs(os.path.join(base_dir, 'train', cl))
    shutil.move(t, os.path.join(base_dir, 'train', cl))

  for v in val:
    if not os.path.exists(os.path.join(base_dir, 'val', cl)):
      os.makedirs(os.path.join(base_dir, 'val', cl))
    shutil.move(v, os.path.join(base_dir, 'val', cl))

    
########################################### JOSN Data split file generation ##############################
  for image in glob.glob(os.path.join(base_dir, 'train', cl) + '/*.jpg'):
      train_list.append(image)
      train_dict.setdefault (cl , []).append(os.path.basename(image)) 
      #print(image)
      
  for image in glob.glob(os.path.join(base_dir, 'val', cl) + '/*.jpg'):
      val_list.append(image)
      val_dict.setdefault (cl , []).append(os.path.basename(image))   
   
dataset_split = {'training' : train_dict , 'validation' : val_dict }

print(dataset_split)
import json
with open('dataset_split.json', 'w') as fp:
      json.dump(dataset_split, fp)


{'training': {'owl': ['IMG_1018.jpg', 'IMG_1022.jpg', 'IMG_1026.jpg', 'IMG_1039.jpg', 'IMG_1044.jpg', 'IMG_1065.jpg', 'IMG_1087.jpg', 'IMG_1088.jpg', 'IMG_1091.jpg', 'IMG_1092.jpg', 'IMG_1112.jpg', 'IMG_1113.jpg', 'IMG_1120.jpg', 'IMG_1145.jpg', 'IMG_1150.jpg', 'IMG_1162.jpg', 'IMG_1170.jpg', 'IMG_1175.jpg', 'IMG_1182.jpg', 'IMG_1223.jpg', 'IMG_1236.jpg', 'IMG_1275.jpg', 'IMG_1286.jpg', 'IMG_1301.jpg', 'IMG_1319.jpg', 'IMG_1326.jpg', 'IMG_1362.jpg', 'IMG_1366.jpg', 'IMG_1373.jpg', 'IMG_1375.jpg', 'IMG_1380.jpg', 'IMG_1388.jpg', 'IMG_1403.jpg', 'IMG_1423.jpg', 'IMG_1425.jpg', 'IMG_1432.jpg', 'IMG_146.jpg', 'IMG_1482.jpg', 'IMG_1598.jpg', 'IMG_1673.jpg', 'IMG_1677.jpg', 'IMG_1694.jpg', 'IMG_1711.jpg', 'IMG_1714.jpg', 'IMG_1715.jpg', 'IMG_1753.jpg', 'IMG_1766.jpg', 'IMG_1779.jpg', 'IMG_1795.jpg', 'IMG_1841.jpg', 'IMG_1920.jpg', 'IMG_1922.jpg', 'IMG_1927.jpg', 'IMG_1954.jpg', 'IMG_1980.jpg', 'IMG_1983.jpg', 'IMG_1993.jpg', 'IMG_1997.jpg', 'IMG_2011.jpg', 'IMG_2048.jpg', 'IMG_258.jpg', 'IMG

In [8]:
round(len(images)*0.8)

0

In [0]:
train_dir = os.path.join(base_dir, 'train') # train directiry 
val_dir = os.path.join(base_dir, 'val')    # validation directory

In the cell below, ImageDataGenerator transform and rescale the images by 255 

flow_from_directory method to apply the above transformation to the images in the training set. batch size, the path to the directory of the training images, the target size for the images, to shuffle the images, the class mode is set to sparse

same applies to validation side but without augmnetation nor shuffling

In [10]:
batch_size = 32
IMG_SHAPE = 224

image_gen_train = ImageDataGenerator(
                   rescale=1./255,               
                   vertical_flip = True,
                   horizontal_flip = True,
                   rotation_range=45,
                   zoom_range=0.5,                                
                   )

train_data_gen = image_gen_train.flow_from_directory(
                                                batch_size=32,
                                                directory=train_dir,
                                                classes = class_list,
                                                shuffle=True,
                                                target_size=(IMG_SHAPE,IMG_SHAPE),
                                                class_mode='sparse',
                                                seed = rand_seed
                                                
                                                )


image_gen_val = ImageDataGenerator(rescale=1./255)
val_data_gen = image_gen_val.flow_from_directory(
                                                 batch_size=16,
                                                 directory=val_dir,
                                                 classes = class_list,
                                                 shuffle=True,
                                                 target_size=(IMG_SHAPE, IMG_SHAPE),
                                                 class_mode='sparse',
                                                 seed = rand_seed
                                                 )

Found 1245 images belonging to 20 classes.
Found 309 images belonging to 20 classes.


convolutional neural network that consists of 6 convolution blocks. Each convolutional block contains a Conv2D layer followed by a max pool layer.

After the 6 convolutional blocks a flatten layer is added and is followed by a fully connected layer with 512 units. The CNN outputs class probabilities based on 20 classes which is done by the softmax activation function. All other layers use a relu activation function. add Dropout layers 0.2 probability is added

In [11]:
model = Sequential()

model.add(Conv2D(filters=8, kernel_size=3,strides=1, padding='same', activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D())
model.add(BatchNormalization())

model.add(Conv2D(filters=16, kernel_size=3,strides=1, padding='same', activation='relu'))
model.add(MaxPooling2D())
model.add(BatchNormalization())

model.add(Conv2D(filters=32, kernel_size=3,strides=1, padding='same', activation='relu'))
model.add(MaxPooling2D())
model.add(BatchNormalization())

model.add(Conv2D(filters=64, kernel_size=3,strides=1, padding='same', activation='relu'))
model.add(MaxPooling2D())
model.add(BatchNormalization())


model.add(Conv2D(filters=128, kernel_size=3,strides=1, padding='same', activation='relu'))
model.add(MaxPooling2D())
model.add(BatchNormalization())


model.add(Conv2D(filters=256, kernel_size=3,strides=1, padding='same', activation='relu'))
model.add(MaxPooling2D())

model.add(BatchNormalization())


model.add(Conv2D(filters=512, kernel_size=3,strides=1, padding='same', activation='relu'))
model.add(MaxPooling2D())

model.add(BatchNormalization())

model.add(Flatten())

model.add(Dense(2056, kernel_initializer=keras.initializers.glorot_uniform(seed=rand_seed) ,activation='relu'))
model.add(Dropout(0.4))

model.add(Dense(1024, kernel_initializer=keras.initializers.glorot_uniform(seed=rand_seed) ,activation='relu'))
model.add(Dropout(0.4))

model.add(Dense(1024, kernel_initializer=keras.initializers.glorot_uniform(seed=rand_seed) ,activation='relu'))
model.add(Dropout(0.4))

model.add(Dense(512, kernel_initializer=keras.initializers.glorot_uniform(seed=rand_seed) ,activation='relu'))
model.add(Dropout(0.4))


model.add(Dense(20, kernel_initializer=keras.initializers.glorot_uniform(seed=rand_seed), activation='softmax'))


model.summary()










Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 224, 224, 8)       224       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 112, 112, 8)       0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 112, 112, 8)       32        
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 112, 112, 16)      1168      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 56, 56, 16)        0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 56, 56, 16)        64     

class wieghts are used to deal with the imbalance in the data set. later on these realtive weights are inputted to the model.fit_generator.

In [0]:
class_weights = {
		0	:	1.052631579	,
		1	:	1.538461538	,
		2	:	1	,
		3	:	1.315789474	,
		4	:	1	,
		5	:	1.428571429	,
		6	:	1.298701299	,
		7	:	1.369863014	,
		8	:	1.333333333	,
		9	:	1.694915254	,
		10	:	1	,
		11	:	1.492537313	,
		12	:	1.428571429	,
		13	:	1.754385965	,
		14	:	1.333333333	,
		15	:	1	,
		16	:	1.298701299	,
		17	:	1.428571429	,
		18	:	1.754385965	,
		19	:	1	
}

The cell below compiles the model. ADAM and SGD optimizers are added for experimenting, the sparse cross entropy function as a loss functn look and training and validation accuracy on each epoch as we train our network

In [13]:
epochs = 100
learning_rate = 0.001
decay_rate = learning_rate / epochs

opt1 = SGD(lr=learning_rate,
            momentum = 0.08, 
            decay = decay_rate
           )

opt2 = Adam(lr=learning_rate, 
            beta_1=0.9, beta_2=0.999, epsilon=None, 
            decay=decay_rate, amsgrad=False)

model.compile(optimizer=opt2,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

The cell below trains model using the fit_generator function. checkpoint are included to capture the best validtion_loss weights and class wieghts take care of the imbalance in the data set  

In [14]:
checkpointer = ModelCheckpoint(filepath='/content/drive/My Drive/weights.best.basic_model.hdf5', 
                               monitor='val_loss',
                               verbose=1, 
                               save_best_only=True
                               )

history = model.fit_generator(
                              train_data_gen,
                              steps_per_epoch=int(np.ceil(train_data_gen.n / float(batch_size))),
                              epochs=100,
                              validation_data=val_data_gen,
                              validation_steps=int(np.ceil(val_data_gen.n / float(batch_size))),
                              callbacks = [checkpointer],
                              class_weight = class_weights,
                              verbose=1
                              )

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/100
39/39 [==============================] - 19s 491ms/step - loss: 4.1058 - acc: 0.0810 - val_loss: 2.8258 - val_acc: 0.1625

Epoch 00001: val_loss improved from inf to 2.82585, saving model to /content/drive/My Drive/weights.best.basic_model.hdf5
Epoch 2/100
39/39 [==============================] - 15s 390ms/step - loss: 3.6561 - acc: 0.1302 - val_loss: 2.7729 - val_acc: 0.1812

Epoch 00002: val_loss improved from 2.82585 to 2.77288, saving model to /content/drive/My Drive/weights.best.basic_model.hdf5
Epoch 3/100
39/39 [==============================] - 17s 433ms/step - loss: 3.5187 - acc: 0.1648 - val_loss: 2.4725 - val_acc: 0.2250

Epoch 00003: val_loss improved from 2.77288 to 2.47248, saving model to /content/drive/My Drive/weights.best.basic_model.hdf5
Epoch 4/100
39/39 [==============================] - 17s 434ms/step - loss: 3.3580 - acc: 0.1995 - val_loss: 2.5823 - val_acc

visulizing the accuracies and losses of both training and validation

In [15]:
acc = history.history['acc']
val_acc = history.history['val_acc']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

KeyError: ignored

The remaining cells do proccesing to convert images to 4D, to predicat on the test data and to generate CSV submission for Kaggle. at the end the predicted image is plotted along with the respective model prediction  

In [0]:
def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(224, 224))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

In [0]:
model.load_weights('/content/drive/My Drive/checkpoints/weights.best.basic_model67.11.hdf5') # load the mode 


In [0]:
def predict_img(img_path):
    img=path_to_tensor(img_path).astype('float32')/255
    # obtain predicted vector
    predicted_vector = model.predict(img)
    # return the index of the max value in the probability vector predicted by the model
    return [np.argmax(predicted_vector)]

In [32]:
from keras.preprocessing import image
(predict_img('/content/drive/My Drive/classification/classification/Classification_Dataset/test/IMG_1035.jpg'))

[5]

In [0]:
def conversion2kaggle (prediction):
    return {
        '[12]'	:	12,
        '[5]'	  :	5,
        '[10]'	:	10,
        '[19]'	:	19,
        '[17]' 	:	17,
        '[18]'  :	18,
        '[16]'  :	16,
        '[13]'  :	13,
        '[2]'	  :	2,
        '[14]'	:	14,
        '[0]'	  :	0,
        '[9]'	  :	9,
        '[15]' 	:	15,
        '[11]'	:	11,
        '[4]'	  :	4,
        '[3]'	  :	3,
        '[1]' 	:	1,
        '[6]'	  :	6,
        '[7]'   :	7,
        '[8]'  	:	8,
    }[prediction]


In [0]:
test = load_files("/content/drive/My Drive/classification/classification/Classification_Dataset/test") #this line takes too much time on colab to execute 5 min 

In [0]:
def create_csv(results, results_dir='./'):

    csv_fname = 'results_'
    csv_fname += datetime.now().strftime('%b%d_%H-%M-%S') + '.csv'

    with open(os.path.join(results_dir, csv_fname), 'w') as f:

        f.write('Id,Category\n')

        for key, value in results.items():
            f.write(key + ',' + str(value) + '\n')

In [36]:
image_list = []
results = {}

for filename in glob.glob('/content/drive/My Drive/classification/classification/Classification_Dataset/test/*.jpg'): 
    image_list.append(filename)


for image_name in image_list:
    results[os.path.basename(image_name)]=conversion2kaggle((str(predict_img(image_name))))

print(results)
create_csv(results)

{'IMG_0.jpg': 19, 'IMG_1001.jpg': 5, 'IMG_299.jpg': 14, 'IMG_1042.jpg': 18, 'IMG_104.jpg': 12, 'IMG_1011.jpg': 15, 'IMG_1031.jpg': 11, 'IMG_1035.jpg': 5, 'IMG_1023.jpg': 3, 'IMG_1037.jpg': 7, 'IMG_1007.jpg': 14, 'IMG_1046.jpg': 19, 'IMG_1073.jpg': 7, 'IMG_1071.jpg': 5, 'IMG_1050.jpg': 5, 'IMG_1059.jpg': 16, 'IMG_106.jpg': 9, 'IMG_105.jpg': 7, 'IMG_1058.jpg': 10, 'IMG_1052.jpg': 11, 'IMG_1072.jpg': 14, 'IMG_1074.jpg': 7, 'IMG_1080.jpg': 8, 'IMG_1090.jpg': 12, 'IMG_1094.jpg': 7, 'IMG_113.jpg': 17, 'IMG_1133.jpg': 5, 'IMG_1097.jpg': 16, 'IMG_1101.jpg': 15, 'IMG_1117.jpg': 10, 'IMG_1099.jpg': 11, 'IMG_1119.jpg': 4, 'IMG_1137.jpg': 14, 'IMG_1084.jpg': 5, 'IMG_1127.jpg': 7, 'IMG_1143.jpg': 11, 'IMG_1147.jpg': 13, 'IMG_1183.jpg': 2, 'IMG_1163.jpg': 14, 'IMG_1179.jpg': 11, 'IMG_1174.jpg': 19, 'IMG_1148.jpg': 16, 'IMG_1158.jpg': 5, 'IMG_1172.jpg': 15, 'IMG_1155.jpg': 8, 'IMG_117.jpg': 12, 'IMG_1187.jpg': 2, 'IMG_119.jpg': 9, 'IMG_1211.jpg': 15, 'IMG_1199.jpg': 6, 'IMG_1194.jpg': 3, 'IMG_1197.jp

In [30]:
def show_img(img_path): 
  img = cv2.imread(img_path)
  cv_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  plt.imshow(cv_rgb)
  plt.show()
  print("     Predicted as {}\n\n\n".format(class_list[conversion2kaggle((str(predict_img(img_path))))])
  ) 

SyntaxError: ignored

In [0]:
for path in image_list:
  show_img(path)